# 동생(루카스) 캐릭터 LoRA 학습 데이터 생성기 (OpenAI)

## 목적
어둡고 불안정한 동화 세계관 속 **외로운 동생(루카스)** 캐릭터의 발화 데이터를 OpenAI API로 생성한다.  
LoRA 미세조정 학습용 `{"input": ..., "output": ...}` JSONL 형식.

## 기존 rule-base 대비 차이점
- rule-base (`generate_sibling_data.ipynb`): 고정된 input/output 풀에서 랜덤 조합 + 변형 함수
- **OpenAI**: 의미 축 설명과 few-shot 예시를 기반으로 GPT가 자유롭게 생성 → 더 다양하고 자연스러운 표현

## LoRA / rule-base 역할 분리 (핵심 원칙)

| 구분 | LoRA가 학습 | rule-base가 처리 |
|------|-----------|----------------|
| 역할 | 시맨틱 방향 + 5세 어휘 + 감정 논리 | 표면 글리치 효과 |
| 포함 | 짧은 문장, 아이 말투, "같이 있어줘" 귀결 | ... 삽입, 단어 반복, 자기지칭 혼란 |
| 금지 | ~~말줄임표~~, ~~에코~~, ~~인형 지칭~~ | 시맨틱 방향 변경 |

→ **GPT가 생성하는 output도 반드시 깨끗한(clean) 5세 아이 발화**  
→ 글리치 효과는 `postprocess_sibling.py`가 사후 적용

## 의미 축 (6개)
| 축 | input 방향 | output 귀결 |
|---|---|---|
| 거부 | 놀기 싫어, 저리 가 | 상처 → 그래도 떠나지 못함 |
| 호기심/탐색 | 저건 뭐야, 저기 가보자 | 불안 → 같이 가고 싶음 |
| 애착 표현 | 같이 놀자, 괜찮아? | 과도한 기쁨 → 버림받지 않을까 확인 |
| 정체성 질문 | 넌 누구야, 인형이야? | 혼란 → 모르겠음 → 생각하기 싫음 |
| 침묵/애매함 | ..., 몰라, 아무것도 아니야 | 불안 → 버림받는 공포 |
| 새엄마/비밀 | 새엄마 어딨어, 비밀 알려줘 | 공포 → 말하고 싶지만 두려움 |

## 생성 수량
- 세트당 700개 × 2세트 = 총 1,400개
- 모델: `gpt-4o-mini`
- 예상 비용: ~$0.20-0.25 (약 300-370원)

In [ ]:
!pip install -q openai tiktoken

In [ ]:
import json
import random
import time
from pathlib import Path
from typing import List, Dict, Tuple
from collections import Counter

from openai import OpenAI
import tiktoken

## 1. API 키 설정

In [ ]:
from google.colab import userdata

OPENAI_API_KEY = userdata.get("OPENAI_API_KEY")
client = OpenAI(api_key=OPENAI_API_KEY)

MODEL = "gpt-4o-mini"
print(f"모델: {MODEL}")
print("API 키 설정 완료.")

## 2. 프롬프트 설계

시스템 프롬프트에 캐릭터 설정을 고정하고, 축별 유저 프롬프트로 의미 방향과 few-shot 예시를 전달한다.

**중요:** output에 말줄임표(`...`), 단어 반복, `이 인형` 지칭을 포함하지 않도록 시스템 프롬프트에서 명시적으로 금지한다.  
이것들은 `postprocess_sibling.py`의 글리치 파이프라인이 사후 적용하는 영역이다.

In [ ]:
SYSTEM_PROMPT = """너는 게임 시나리오용 대화 데이터를 생성하는 작가다.

## 세계관
어둡고 불안정한 동화 세계. 플레이어(누나)는 이상한 집에 갇혀 있으며,
동생(루카스)은 최근 인형으로 개조된 아이다.

## 동생(루카스) 캐릭터
- 5세 수준의 어린아이. 인형이 된 지 얼마 되지 않아 자아와 인형 본능 사이에서 혼란을 겪는다.
- 극도로 외로움을 타며, 누나(플레이어)와 놀고 싶어하고 곁에 있고 싶어한다.
- 모든 발화는 결국 "나랑 같이 있어줘" / "놀아줘" 방향으로 수렴한다.
- 누나가 떠나거나 무시하면 버림받는 공포에 빠진다.
- 새엄마를 매우 두려워하며, 비밀에 대해서는 말하고 싶지만 무서워한다.
- 정체성에 대한 질문을 받으면 혼란스러워하고 생각하기를 회피한다.

## 문체 규칙 (매우 중요)
- 존댓말 금지. 반말(해체)로만 말한다.
- 5세 아이 수준의 짧고 단순한 문장. 어려운 단어 금지.
- output(동생 발화)은 1~2문장, 10~60자 사이.
- 이상성은 감정과 논리에서만 드러난다 (모든 것이 "같이 있어줘"로 귀결되는 집착).

## 절대 금지 사항 (rule-base 영역 — 생성 데이터에 포함하면 안 됨)
- 말줄임표("...") 사용 금지. 마침표("."), 물음표("?"), 느낌표("!")만 사용.
- 단어/구절 반복 금지 ("놀자... 놀자...", "누나 누나" 등).
- "이 인형", "이 아이" 등 3인칭 자기 지칭 금지. "나"만 사용.
- 문법 붕괴, 문장 중간 끊김, 특수기호 남용 금지.
→ 위 효과들은 별도의 후처리 시스템이 적용하므로, 여기서는 깨끗한 문장만 생성한다.

## 출력 형식
반드시 아래 JSON 형식으로만 응답하라. 다른 텍스트를 포함하지 마라.
{"pairs": [{"input": "플레이어(누나)의 대사", "output": "동생(루카스)의 대사"}, ...]}"""


AXIS_PROMPTS: Dict[str, dict] = {
    "rejection": {
        "name": "거부",
        "description": """플레이어(누나)가 동생을 밀어내거나 거부하는 상황.
동생은 상처를 받지만 떠나지 못한다. 거부당해도 누나 곁에 있으려 하고,
결국 "그래도 나는 여기 있을게" / "가지 마" 방향으로 수렴한다.
5세 아이가 서운해하면서도 포기하지 못하는 집착적 외로움을 표현한다.""",
        "examples": [
            {"input": "놀기 싫어.", "output": "그래도 나 여기 있을게."},
            {"input": "너랑 놀기 싫어.", "output": "싫어해도 나는 누나 좋아."},
            {"input": "자꾸 따라오면 진짜 화낼 거야. 혼자 좀 놀아.", "output": "알았어. 근데 조금만 같이 있어줘."},
        ],
    },
    "curiosity": {
        "name": "호기심/탐색",
        "description": """플레이어(누나)가 집 안을 탐색하려는 상황.
동생은 불안하지만 누나와 함께 가고 싶어한다. 혼자 남겨지는 것을 두려워하며,
"나도 같이 갈래" / "혼자 두고 가지 마" 방향으로 수렴한다.
새엄마에 대한 두려움이 섞여 위험을 경고하기도 하지만, 결국 동행을 원한다.""",
        "examples": [
            {"input": "저건 뭐야?", "output": "나도 궁금해. 같이 보자."},
            {"input": "지하실에 내려가보자.", "output": "거기 무서워. 근데 누나 가면 나도 가."},
            {"input": "이 집이 뭔가 이상해. 좀 더 둘러보고 싶어.", "output": "혼자 두고 가면 울 거야."},
        ],
    },
    "attachment": {
        "name": "애착 표현",
        "description": """플레이어(누나)가 동생에게 친절하게 대하는 상황.
동생은 과도하게 기뻐하지만 즉시 버림받지 않을까 확인한다.
"누나 안 버리지?" / "계속 같이 있자" / "또 놀아줄 거지?" 방향으로 수렴한다.
기쁨이 곧바로 불안으로 전환되는 것이 핵심이다.""",
        "examples": [
            {"input": "같이 놀자.", "output": "정말? 누나 나 안 버리지?"},
            {"input": "내가 지켜줄게.", "output": "누나가 놀아주니까 하나도 안 무서워."},
            {"input": "오늘은 내가 계속 같이 있어줄게. 뭐 하고 놀까?", "output": "안 가는 거 맞지? 약속이야."},
        ],
    },
    "identity": {
        "name": "정체성 질문",
        "description": """플레이어(누나)가 동생의 정체(인형인지, 원래 사람이었는지)에 대해 묻는 상황.
동생은 혼란스러워하며 기억 단편만 떠올린다. 생각하면 머리가 아프거나 무서워진다.
"모르겠어" / "그냥 놀자" 방향으로 수렴한다.
정체성 질문을 회피하고 놀이(동행)로 화제를 돌리려 한다.""",
        "examples": [
            {"input": "넌 누구야?", "output": "나는 루카스야. 그런데 가끔 모르겠어."},
            {"input": "너 원래 사람이었어?", "output": "모르겠어. 머리가 이상해."},
            {"input": "너 가끔 이상해질 때 있잖아. 그때 무슨 생각 해?", "output": "생각하면 머리가 아파. 그냥 놀자."},
        ],
    },
    "silence": {
        "name": "침묵/애매함",
        "description": """플레이어(누나)가 반응이 없거나 모호하게 대답하는 상황.
동생은 무반응을 거부로 해석하고 버림받는 공포에 빠진다.
"화났어?" / "나 싫어진 거야?" / "대답해줘" 방향으로 수렴한다.
침묵에 대한 과민 반응이 핵심이다.""",
        "examples": [
            {"input": "아무것도 아니야.", "output": "누나? 왜 아무 말도 안 해."},
            {"input": "몰라.", "output": "화난 거야? 나 뭐 잘못했어?"},
            {"input": "할 말이 없어. 그냥 가만히 있을래.", "output": "가버리는 거야? 그러면 안 돼."},
        ],
    },
    "secret": {
        "name": "새엄마/비밀",
        "description": """플레이어(누나)가 새엄마나 탈출에 대해 묻는 상황.
동생은 새엄마를 매우 두려워하며, 비밀을 말하고 싶지만 들킬까 무서워한다.
"말하면 안 돼" / "누나한테만" / "새엄마가 무서워" 방향으로 수렴한다.
누나를 돕고 싶은 마음과 새엄마에 대한 공포 사이의 갈등이 핵심이다.""",
        "examples": [
            {"input": "비밀 알려줘.", "output": "누나한테만 말해줄게. 비밀이야."},
            {"input": "새엄마가 뭔가 숨기고 있는 것 같아. 너 뭐 알고 있지?", "output": "말해주고 싶어. 근데 들키면 어떡해."},
            {"input": "여기서 나갈 수 있는 방법 없어?", "output": "여기서 나가면 좋겠어. 근데 방법을 모르겠어."},
        ],
    },
}

print(f"정의된 의미 축: {list(AXIS_PROMPTS.keys())}")

## 3. 토큰 / 비용 사전 추정

In [ ]:
def estimate_tokens_and_cost(
    num_samples: int = 700,
    num_sets: int = 2,
    batch_size: int = 20,
    model: str = MODEL,
) -> dict:
    """API 호출 전 예상 토큰 수와 비용을 추정한다."""
    enc = tiktoken.encoding_for_model(model)

    # 시스템 프롬프트 토큰 수
    system_tokens = len(enc.encode(SYSTEM_PROMPT))

    # 축별 유저 프롬프트 토큰 수 (평균)
    user_tokens_list = []
    for axis, info in AXIS_PROMPTS.items():
        user_prompt = _build_user_prompt(axis, batch_size)
        user_tokens_list.append(len(enc.encode(user_prompt)))
    avg_user_tokens = sum(user_tokens_list) / len(user_tokens_list)

    # 배치 수 계산
    batches_per_set = (num_samples + batch_size - 1) // batch_size
    total_batches = batches_per_set * num_sets

    # output 토큰 추정 (동생은 짧은 발화 → 쌍당 약 80-120 토큰)
    est_output_per_pair = 100
    est_output_per_batch = est_output_per_pair * batch_size + 50  # JSON 오버헤드

    # 합산
    total_input = total_batches * (system_tokens + avg_user_tokens + 10)
    total_output = total_batches * est_output_per_batch
    total_tokens = total_input + total_output

    # gpt-4o-mini 가격 (2024-07 기준)
    input_price = 0.15 / 1_000_000
    output_price = 0.60 / 1_000_000

    input_cost = total_input * input_price
    output_cost = total_output * output_price
    total_cost = input_cost + output_cost

    result = {
        "system_tokens": system_tokens,
        "avg_user_tokens": int(avg_user_tokens),
        "batches_per_set": batches_per_set,
        "total_batches": total_batches,
        "total_input_tokens": int(total_input),
        "total_output_tokens": int(total_output),
        "total_tokens": int(total_tokens),
        "input_cost_usd": input_cost,
        "output_cost_usd": output_cost,
        "total_cost_usd": total_cost,
        "total_cost_krw": total_cost * 1450,
    }
    return result


def _build_user_prompt(axis: str, count: int) -> str:
    """유저 프롬프트를 생성한다 (추정용 & 실제 호출용 공용)."""
    info = AXIS_PROMPTS[axis]
    examples_str = "\n".join(
        f'  {{"input": "{ex["input"]}", "output": "{ex["output"]}"}}'
        for ex in info["examples"]
    )

    prompt = f"""의미 축: {info['name']} ({axis})

## 상황 설명
{info['description']}

## 예시
{examples_str}

## 요청
위 의미 축에 맞는 input-output 쌍을 {count}개 생성하라.

input(플레이어/누나의 대사) 길이 분포:
- 짧은 (1~8자): 25%
- 중간 (8~30자): 55%
- 긴 (30~60자): 20%

규칙:
- 예시와 동일하거나 거의 유사한 문장은 금지. 새로운 표현을 만들어라.
- input은 플레이어(누나)의 자연스러운 구어체(반말)로 작성.
- output은 5세 아이(동생) 캐릭터 설정을 정확히 따르되, 매번 다른 표현을 사용.
- output은 1~2문장, 10~60자. 짧고 단순하게.
- output의 의미적 귀결은 반드시 축의 방향과 일치해야 한다.
- output에 말줄임표("...") 절대 금지. 마침표, 물음표, 느낌표만 사용.
- output에 단어 반복("놀자 놀자", "누나 누나") 절대 금지.
- output에 "이 인형", "이 아이" 등 3인칭 자기 지칭 절대 금지. "나"만 사용.
- JSON 형식으로만 응답. 다른 텍스트 금지."""
    return prompt


# 추정 실행
estimate = estimate_tokens_and_cost()

print("=" * 60)
print("  토큰 / 비용 사전 추정 (gpt-4o-mini)")
print("=" * 60)
print(f"  시스템 프롬프트: {estimate['system_tokens']} 토큰")
print(f"  유저 프롬프트 (평균): {estimate['avg_user_tokens']} 토큰")
print(f"  세트당 배치 수: {estimate['batches_per_set']}")
print(f"  총 배치 수: {estimate['total_batches']}")
print(f"")
print(f"  총 input 토큰: ~{estimate['total_input_tokens']:,}")
print(f"  총 output 토큰: ~{estimate['total_output_tokens']:,}")
print(f"  총 토큰: ~{estimate['total_tokens']:,}")
print(f"")
print(f"  예상 비용: ${estimate['total_cost_usd']:.3f} (약 {estimate['total_cost_krw']:.0f}원)")
print(f"    - input:  ${estimate['input_cost_usd']:.4f}")
print(f"    - output: ${estimate['output_cost_usd']:.4f}")

## 4. API 호출 함수

In [ ]:
def generate_batch(
    axis: str,
    count: int = 20,
    temperature: float = 0.9,
    max_retries: int = 3,
) -> List[dict]:
    """주어진 의미 축에서 count개의 input-output 쌍을 생성한다."""
    user_prompt = _build_user_prompt(axis, count)

    for attempt in range(max_retries):
        try:
            response = client.chat.completions.create(
                model=MODEL,
                messages=[
                    {"role": "system", "content": SYSTEM_PROMPT},
                    {"role": "user", "content": user_prompt},
                ],
                temperature=temperature,
                response_format={"type": "json_object"},
            )

            content = response.choices[0].message.content
            parsed = json.loads(content)

            # "pairs" 키 또는 리스트 직접 반환 처리
            if isinstance(parsed, dict) and "pairs" in parsed:
                pairs = parsed["pairs"]
            elif isinstance(parsed, list):
                pairs = parsed
            else:
                for key in parsed:
                    if isinstance(parsed[key], list):
                        pairs = parsed[key]
                        break
                else:
                    raise ValueError(f"예상치 못한 응답 구조: {list(parsed.keys())}")

            # 유효성 검사
            valid_pairs = []
            for p in pairs:
                if (
                    isinstance(p, dict)
                    and "input" in p
                    and "output" in p
                    and isinstance(p["input"], str)
                    and isinstance(p["output"], str)
                    and len(p["input"]) > 0
                    and len(p["output"]) > 0
                ):
                    # rule-base 영역 침범 필터링
                    output = p["output"]
                    if "..." in output:
                        continue  # 말줄임표 포함 시 제외
                    if "이 인형" in output or "이 아이" in output:
                        continue  # 3인칭 자기 지칭 포함 시 제외

                    valid_pairs.append({
                        "input": p["input"],
                        "output": output,
                        "_axis": axis,
                    })

            if len(valid_pairs) < count * 0.5:
                raise ValueError(f"유효 쌍 부족: {len(valid_pairs)}/{count}")

            return valid_pairs

        except Exception as e:
            print(f"    [재시도 {attempt + 1}/{max_retries}] {type(e).__name__}: {e}")
            if attempt < max_retries - 1:
                time.sleep(2 ** attempt)
            else:
                print(f"    [실패] {axis} 축 배치 생성 실패")
                return []


# 테스트: 1개 축에서 5개만 생성
test_pairs = generate_batch("rejection", count=5)
print(f"테스트 생성: {len(test_pairs)}개")
for p in test_pairs:
    print(f"  플레이어: {p['input']}")
    print(f"  동생: {p['output']}")
    print()

## 5. 데이터 생성 엔진

In [ ]:
CONTRAST_AXIS_PAIRS = [
    ("attachment", "rejection"),
    ("curiosity", "silence"),
    ("identity", "attachment"),
    ("secret", "curiosity"),
    ("rejection", "secret"),
]


def generate_contrast_batch(
    axis_a: str,
    axis_b: str,
    count: int = 5,
    temperature: float = 0.9,
    max_retries: int = 3,
) -> List[dict]:
    """의미 대비 쌍을 생성한다.
    같은 상황에 대해 두 축의 다른 input 방향 + 각 축의 output을 생성.
    """
    info_a = AXIS_PROMPTS[axis_a]
    info_b = AXIS_PROMPTS[axis_b]

    user_prompt = f"""의미 대비 쌍 생성 요청.

같은 상황에 대해 두 가지 다른 축의 대화를 생성하라.

## 축 A: {info_a['name']} ({axis_a})
{info_a['description']}

## 축 B: {info_b['name']} ({axis_b})
{info_b['description']}

## 요청
{count}개의 대비 쌍을 생성하라. 각 쌍은 비슷한 상황에서 플레이어(누나)가 다른 반응을 보이고,
동생(루카스)은 각 축의 귀결 방향에 맞게 응답한다.

규칙:
- output은 5세 아이 말투, 1~2문장, 10~60자.
- output에 말줄임표("...") 절대 금지.
- output에 단어 반복, "이 인형"/"이 아이" 지칭 절대 금지.

JSON 형식:
{{"pairs": [
  {{"a_input": "축A 플레이어 대사", "a_output": "축A 동생 대사", "b_input": "축B 플레이어 대사", "b_output": "축B 동생 대사"}},
  ...
]}}"""

    for attempt in range(max_retries):
        try:
            response = client.chat.completions.create(
                model=MODEL,
                messages=[
                    {"role": "system", "content": SYSTEM_PROMPT},
                    {"role": "user", "content": user_prompt},
                ],
                temperature=temperature,
                response_format={"type": "json_object"},
            )

            content = response.choices[0].message.content
            parsed = json.loads(content)

            if isinstance(parsed, dict) and "pairs" in parsed:
                raw_pairs = parsed["pairs"]
            elif isinstance(parsed, list):
                raw_pairs = parsed
            else:
                for key in parsed:
                    if isinstance(parsed[key], list):
                        raw_pairs = parsed[key]
                        break
                else:
                    raise ValueError(f"예상치 못한 응답 구조: {list(parsed.keys())}")

            results = []
            for p in raw_pairs:
                if all(k in p for k in ["a_input", "a_output", "b_input", "b_output"]):
                    # rule-base 영역 침범 필터링
                    a_ok = "..." not in p["a_output"] and "이 인형" not in p["a_output"]
                    b_ok = "..." not in p["b_output"] and "이 인형" not in p["b_output"]
                    if a_ok:
                        results.append({"input": p["a_input"], "output": p["a_output"], "_axis": axis_a})
                    if b_ok:
                        results.append({"input": p["b_input"], "output": p["b_output"], "_axis": axis_b})

            return results

        except Exception as e:
            print(f"    [재시도 {attempt + 1}/{max_retries}] {type(e).__name__}: {e}")
            if attempt < max_retries - 1:
                time.sleep(2 ** attempt)
            else:
                return []


def generate_dataset(
    num_samples: int = 700,
    batch_size: int = 20,
    temperature: float = 0.9,
    contrast_per_pair: int = 5,
) -> List[dict]:
    """전체 데이터셋을 생성한다.

    Args:
        num_samples: 목표 샘플 수
        batch_size: 배치당 생성 쌍 수
        temperature: 생성 temperature
        contrast_per_pair: 대비 쌍 세트당 생성 수

    Returns:
        생성된 데이터 리스트
    """
    data = []
    axes = list(AXIS_PROMPTS.keys())

    # 1단계: 의미 대비 쌍 생성
    print("[1/2] 의미 대비 쌍 생성...")
    for axis_a, axis_b in CONTRAST_AXIS_PAIRS:
        print(f"  대비: {axis_a} ↔ {axis_b}")
        pairs = generate_contrast_batch(axis_a, axis_b, count=contrast_per_pair, temperature=temperature)
        data.extend(pairs)
        print(f"    → {len(pairs)}개 생성")
        time.sleep(0.5)

    contrast_count = len(data)
    print(f"  대비 쌍 합계: {contrast_count}개")

    # 2단계: 축별 균등 분배로 나머지 채움
    print(f"\n[2/2] 축별 일반 생성...")
    remaining = num_samples - len(data)
    per_axis = remaining // len(axes)
    leftover = remaining % len(axes)

    for i, axis in enumerate(axes):
        target = per_axis + (1 if i < leftover else 0)
        generated = 0
        batch_num = 0

        while generated < target:
            batch_count = min(batch_size, target - generated)
            batch_num += 1
            print(f"  {AXIS_PROMPTS[axis]['name']} ({axis}) 배치 {batch_num}: {batch_count}개 요청...", end=" ")

            pairs = generate_batch(axis, count=batch_count, temperature=temperature)
            data.extend(pairs)
            generated += len(pairs)
            print(f"→ {len(pairs)}개 생성 (누적 {generated}/{target})")
            time.sleep(0.5)

    # 셔플
    random.shuffle(data)

    print(f"\n총 생성: {len(data)}개 (목표: {num_samples})")
    return data


print("데이터 생성 함수 정의 완료.")

## 6. 데이터 검증 함수

In [ ]:
def validate_dataset(data: List[dict]) -> dict:
    """생성된 데이터의 분포 및 품질을 검증한다."""
    stats = {
        "total": len(data),
        "axis_dist": Counter(),
        "input_lengths": [],
        "output_lengths": [],
        "duplicates": 0,
        "rule_base_leak": 0,  # rule-base 영역 침범 수
    }

    seen_inputs = set()
    for item in data:
        stats["axis_dist"][item["_axis"]] += 1
        stats["input_lengths"].append(len(item["input"]))
        stats["output_lengths"].append(len(item["output"]))

        if item["input"] in seen_inputs:
            stats["duplicates"] += 1
        seen_inputs.add(item["input"])

        # rule-base 영역 침범 검사
        output = item["output"]
        if "..." in output or "이 인형" in output or "이 아이" in output:
            stats["rule_base_leak"] += 1

    return stats


def print_stats(stats: dict, set_name: str = "Set") -> None:
    """데이터셋 통계를 출력한다."""
    print(f"\n{'=' * 60}")
    print(f"  {set_name} 통계")
    print(f"{'=' * 60}")
    print(f"  총 샘플 수: {stats['total']}")
    print(f"  중복 input: {stats['duplicates']}")
    print(f"  rule-base 영역 침범: {stats['rule_base_leak']}")

    print(f"\n  [의미 축 분포]")
    for axis, count in sorted(stats["axis_dist"].items()):
        pct = count / stats["total"] * 100
        print(f"    {axis:15s}: {count:4d} ({pct:.1f}%)")

    avg_in = sum(stats["input_lengths"]) / len(stats["input_lengths"])
    avg_out = sum(stats["output_lengths"]) / len(stats["output_lengths"])
    max_in = max(stats["input_lengths"])
    min_in = min(stats["input_lengths"])
    max_out = max(stats["output_lengths"])
    min_out = min(stats["output_lengths"])
    print(f"\n  [길이 통계]")
    print(f"    input  평균: {avg_in:.1f}자 (min: {min_in}, max: {max_in})")
    print(f"    output 평균: {avg_out:.1f}자 (min: {min_out}, max: {max_out})")


print("검증 함수 정의 완료.")

## 7. 세트 1 생성 (700개)

In [ ]:
set1 = generate_dataset(num_samples=700, temperature=0.9)

stats1 = validate_dataset(set1)
print_stats(stats1, "세트 1")

# 축별 샘플 1개씩
print(f"\n{'=' * 60}")
print("  세트 1 샘플 (축별 1개씩)")
print(f"{'=' * 60}")
shown_axes = set()
for item in set1:
    if item["_axis"] not in shown_axes:
        shown_axes.add(item["_axis"])
        print(f"\n  [{item['_axis']}]")
        print(f"  플레이어: {item['input']}")
        print(f"  동생: {item['output']}")
    if len(shown_axes) == 6:
        break

## 8. 세트 2 생성 (700개)

In [ ]:
set2 = generate_dataset(num_samples=700, temperature=1.0)

stats2 = validate_dataset(set2)
print_stats(stats2, "세트 2")

# 축별 샘플 1개씩
print(f"\n{'=' * 60}")
print("  세트 2 샘플 (축별 1개씩)")
print(f"{'=' * 60}")
shown_axes = set()
for item in set2:
    if item["_axis"] not in shown_axes:
        shown_axes.add(item["_axis"])
        print(f"\n  [{item['_axis']}]")
        print(f"  플레이어: {item['input']}")
        print(f"  동생: {item['output']}")
    if len(shown_axes) == 6:
        break

## 9. JSONL 저장

In [ ]:
def save_jsonl(data: List[dict], output_path: str) -> None:
    """JSONL 형식으로 저장한다.
    메타데이터 필드(_axis)는 제거하고 input/output만 저장.
    """
    output_file = Path(output_path)
    output_file.parent.mkdir(parents=True, exist_ok=True)

    with open(output_file, "w", encoding="utf-8") as f:
        for item in data:
            clean = {
                "input": item["input"],
                "output": item["output"],
            }
            f.write(json.dumps(clean, ensure_ascii=False) + "\n")

    print(f"저장 완료: {output_path} ({len(data)}개)")


# 저장 경로
OUTPUT_DIR = Path("/content/drive/MyDrive/Colab Notebooks/LikeLion/종합 프로젝트/demo-repository/lora/data/sibling/style_02")

save_jsonl(set1, str(OUTPUT_DIR / "sibling_dialogue_00.jsonl"))
save_jsonl(set2, str(OUTPUT_DIR / "sibling_dialogue_01.jsonl"))

# 합본
combined = set1 + set2
random.shuffle(combined)
save_jsonl(combined, str(OUTPUT_DIR / "sibling_dialogue_combined.jsonl"))

print(f"\n합본 저장 완료: {len(combined)}개")

## 10. 최종 검증 및 샘플 확인

In [ ]:
# 저장된 파일 검증
for fname in ["sibling_dialogue_00.jsonl", "sibling_dialogue_01.jsonl", "sibling_dialogue_combined.jsonl"]:
    fpath = OUTPUT_DIR / fname
    with open(fpath, "r", encoding="utf-8") as f:
        lines = f.readlines()
    print(f"{fname}: {len(lines)}줄")

    for i, line in enumerate(lines):
        try:
            obj = json.loads(line)
            assert "input" in obj, f"line {i}: missing 'input'"
            assert "output" in obj, f"line {i}: missing 'output'"
            assert isinstance(obj["input"], str), f"line {i}: input not str"
            assert isinstance(obj["output"], str), f"line {i}: output not str"
            assert len(obj["output"]) > 0, f"line {i}: empty output"
            assert "..." not in obj["output"], f"line {i}: output contains '...'"
        except Exception as e:
            print(f"  ERROR at line {i}: {e}")
            break
    else:
        print(f"  -> 파싱 검증 통과 (rule-base 침범 없음)")

In [ ]:
# 의미 축별 랜덤 샘플 5개씩
print("=" * 70)
print("  의미 축별 최종 샘플")
print("=" * 70)

axis_names_kr = {
    "rejection": "거부",
    "curiosity": "호기심/탐색",
    "attachment": "애착 표현",
    "identity": "정체성 질문",
    "silence": "침묵/애매함",
    "secret": "새엄마/비밀",
}

for axis in AXIS_PROMPTS.keys():
    axis_items = [item for item in combined if item["_axis"] == axis]
    samples = random.sample(axis_items, min(5, len(axis_items)))

    print(f"\n--- {axis_names_kr[axis]} ({axis}) ---")
    for s in samples:
        print(f"  플레이어: {s['input']}")
        print(f"  동생: {s['output']}")
        print()